In [6]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset
from torch.optim import Adam
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
from numpy import genfromtxt
from sklearn.preprocessing import LabelEncoder



In [7]:
# Load Data
x = genfromtxt('../Data/WISDM_x.csv', delimiter=',')
y_df = pd.read_csv('../Data/WISDM_y.csv')
y = y_df.values.flatten()  # Flatten if y is 2D

# Encode labels
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# Function to create time series dataset
def create_series(x, y, timestep, overlap):
    slide_step = int(timestep * (1 - overlap))
    data_num = int((len(x) / slide_step) - 1)
    dataset = np.ndarray(shape=(data_num, timestep, x.shape[1]))
    labels = []

    for i in range(data_num):
        labels.append(y[slide_step * (i + 1) - 1])
        for j in range(timestep):
            dataset[i, j, :] = x[slide_step * i + j, :]

    return dataset, np.array(labels)

# Create time series
timestep = 16  # Replace with your value
overlap = 0.5  # Replace with your value
X_series, y_series = create_series(x, y_encoded, timestep, overlap)

In [11]:
# Split into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X_series, y_series, test_size=0.2, random_state=42)
print(f'X_train shape:{X_train.shape}, X_test shape:{X_test.shape}, y_train shape:{y_train.shape}, y_test shape:{y_test.shape}')

X_train shape:(109820, 16, 3), X_test shape:(27455, 16, 3), y_train shape:(109820,), y_test shape:(27455,)


In [10]:
class MyModel(nn.Module):
    def __init__(self, n_input, n_steps):
        super(MyModel, self).__init__()
        self.conv1 = nn.Conv1d(in_channels=n_input, out_channels=256, kernel_size=3)
        self.conv2 = nn.Conv1d(in_channels=256, out_channels=128, kernel_size=3)
        self.conv3 = nn.Conv1d(in_channels=128, out_channels=64, kernel_size=3)
        self.conv4 = nn.Conv1d(in_channels=64, out_channels=32, kernel_size=3)
        self.flatten = nn.Flatten()
        self.fc = nn.Linear(32 * 8, 6)  # Adjusted for the output from the last Conv1d layer
        self.dropout = nn.Dropout(0.2)
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        x = self.dropout(torch.relu(self.conv1(x)))
        x = self.dropout(torch.relu(self.conv2(x)))
        x = self.dropout(torch.relu(self.conv3(x)))
        x = self.dropout(torch.relu(self.conv4(x)))
        x = self.flatten(x)
        x = self.fc(x)
        return self.softmax(x)

# Example instantiation of the model
model = MyModel(n_input=3, n_steps=16)


In [12]:

# Convert arrays to PyTorch Tensors
X_train = torch.tensor(X_train, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.long)  # Assuming y_train is class labels for classification
X_test = torch.tensor(X_test, dtype=torch.float32)
y_test = torch.tensor(y_test, dtype=torch.long)

# Creating TensorDatasets
train_dataset = TensorDataset(X_train, y_train)
test_dataset = TensorDataset(X_test, y_test)

# Creating DataLoaders
batch_size = 64
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)


In [15]:
import torch.optim as optim

# Setting up the device (GPU/CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
epochs = 20
for epoch in range(epochs):
    model.train()  # Set the model to training mode
    train_loss = 0.0

    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        inputs = inputs.view(-1, 3, 16)  # Reshape input to match model expectations

        # Zero the parameter gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, labels)

        # Backward pass and optimize
        loss.backward()
        optimizer.step()

        train_loss += loss.item() * inputs.size(0)

    # Validation phase
    model.eval()  # Set the model to evaluation mode
    val_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            inputs = inputs.view(-1, 3, 16)

            outputs = model(inputs)
            loss = criterion(outputs, labels)
            val_loss += loss.item() * inputs.size(0)

            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    # Calculate average losses and accuracy
    train_loss = train_loss / len(train_loader.dataset)
    val_loss = val_loss / len(test_loader.dataset)
    val_accuracy = 100 * correct / total

    # Print training/validation statistics
    print(f'Epoch: {epoch+1}/{epochs} \tTraining Loss: {train_loss:.4f} \tValidation Loss: {val_loss:.4f} \tValidation Accuracy: {val_accuracy:.2f}%')

print('Training complete')


Epoch: 1/20 	Training Loss: 1.2956 	Validation Loss: 1.2914 	Validation Accuracy: 75.18%
Epoch: 2/20 	Training Loss: 1.3012 	Validation Loss: 1.2932 	Validation Accuracy: 75.02%
Epoch: 3/20 	Training Loss: 1.2983 	Validation Loss: 1.2874 	Validation Accuracy: 75.61%
Epoch: 4/20 	Training Loss: 1.3037 	Validation Loss: 1.3023 	Validation Accuracy: 74.12%
Epoch: 5/20 	Training Loss: 1.3084 	Validation Loss: 1.2967 	Validation Accuracy: 74.69%
Epoch: 6/20 	Training Loss: 1.3030 	Validation Loss: 1.3070 	Validation Accuracy: 73.65%
Epoch: 7/20 	Training Loss: 1.3092 	Validation Loss: 1.2960 	Validation Accuracy: 74.76%
Epoch: 8/20 	Training Loss: 1.3044 	Validation Loss: 1.3109 	Validation Accuracy: 73.27%
Epoch: 9/20 	Training Loss: 1.3017 	Validation Loss: 1.3039 	Validation Accuracy: 73.97%
Epoch: 10/20 	Training Loss: 1.3105 	Validation Loss: 1.3079 	Validation Accuracy: 73.57%
Epoch: 11/20 	Training Loss: 1.3112 	Validation Loss: 1.2980 	Validation Accuracy: 74.56%
Epoch: 12/20 	Train

In [17]:
# Test phase - after training is complete
model.eval()  # Set the model to evaluation mode
test_loss = 0.0
correct = 0
total = 0

with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        inputs = inputs.view(-1, 3, 16)

        outputs = model(inputs)
        loss = criterion(outputs, labels)
        test_loss += loss.item() * inputs.size(0)

        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

# Calculate average loss and accuracy
test_loss = test_loss / len(test_loader.dataset)
test_accuracy = 100 * correct / total

# Print test statistics
print(f'Test Loss: {test_loss:.4f} \tTest Accuracy: {test_accuracy:.2f}%')


Test Loss: 1.3202 	Test Accuracy: 72.34%


In [21]:
model_path = "/Users/sandeep/Desktop/BUCourses/Project/saved_models/Pytorch/CNN_base.pth"
torch.save(model.state_dict(), model_path)


In [27]:
def compute_metrics_base(model, test_loader, model_path):
    """
    Compute the accuracy of the PyTorch model.

    :param model: PyTorch model.
    :param test_loader: DataLoader for the test dataset.
    :param model_path: Path to the PyTorch model file.
    :return: None
    """

    model.eval()
    correct = 0
    total = 0
    all_predicted = []
    all_labels = []

    with torch.no_grad():
        for inputs, labels in test_loader:
            # Reshaping and predictions
            inputs = inputs.view(-1, 3, 16)  # Adjust the reshape as per your model input
            outputs = model(inputs)
            _, predicted_labels = torch.max(outputs, 1)

            # Aggregate labels and predictions
            all_predicted.extend(predicted_labels.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

            # Calculating accuracy
            correct += (predicted_labels == labels).sum().item()
            total += labels.size(0)

    # Compute accuracy
    accuracy = correct / total
    print(f'Accuracy on the test set: {accuracy:.2%}')

    # Model size
    model_size_bytes = Path(model_path).stat().st_size
    model_size_kb = model_size_bytes / 1024
    print(f"Size of the model: {model_size_kb:.2f} KB")


In [24]:
def measure_cpu_utilization_and_run(func, *args, **kwargs):
    """
    Measure CPU utilization while running a function.

    Parameters:
        func (function): The function to be executed.
        *args: Arguments to be passed to func.
        **kwargs: Keyword arguments to be passed to func.

    Returns:
        float: CPU utilization percentage during the execution of func.
        float: The elapsed time during the execution of func.
        any: The result of func execution.
    """
    
    # Measure CPU utilization before execution
    cpu_percent_before = psutil.cpu_percent(interval=None)

    # Record the start time
    start_time = time.time()

    # Execute the function and store its result
    result = func(*args, **kwargs)

    # Record the end time
    end_time = time.time()

    # Measure CPU utilization after execution
    cpu_percent_after = psutil.cpu_percent(interval=None)

    # Calculate elapsed time and average CPU utilization
    elapsed_time = end_time - start_time
    average_cpu_utilization = (cpu_percent_before + cpu_percent_after) / 2

    return average_cpu_utilization, elapsed_time, result



In [28]:
# Measure CPU usage and inference time
cpu_usage, inference_time, _ = measure_cpu_utilization_and_run(compute_metrics_base, model, test_loader, model_path)

print(f'CPU usage during inference: {cpu_usage:.2f}%')
print(f'Inference time: {inference_time:.4f} seconds')


Accuracy on the test set: 72.34%
Size of the model: 524.57 KB
CPU usage during inference: 40.60%
Inference time: 4.8561 seconds


In [31]:
import torch.quantization
torch.backends.quantized.engine = 'qnnpack'


In [32]:

quantized_model = torch.quantization.quantize_dynamic(
    model,  # the original model
    {nn.Linear, nn.Conv1d},  # specify which layer types to quantize
    dtype=torch.qint8  # the target data type for quantized weights
)


In [35]:
# Save the quantized model
torch.save(quantized_model.state_dict(), '/Users/sandeep/Desktop/BUCourses/Project/saved_models/Pytorch/CNN_Quantized.pth')

# Use it for inference
quantized_model.eval()
with torch.no_grad():
    for inputs, labels in test_loader:
        inputs = inputs.view(-1, 3, 16)  # Reshape if needed, based on your model's requirement
        outputs = quantized_model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print(f'Accuracy of the quantized model on the test dataset: {accuracy:.2f}%')


[W qlinear_dynamic.cpp:247] Warning: Currently, qnnpack incorrectly ignores reduce_range when it is set to true; this may change in a future release. (function operator())


Accuracy of the quantized model on the test dataset: 72.33%


In [37]:
# Measure CPU usage and inference time
quantized_model_path = '/Users/sandeep/Desktop/BUCourses/Project/saved_models/Pytorch/CNN_Quantized.pth'
cpu_usage, inference_time, _ = measure_cpu_utilization_and_run(compute_metrics_base, quantized_model, test_loader, quantized_model_path)

print(f'CPU usage during inference: {cpu_usage:.2f}%')
print(f'Inference time: {inference_time:.4f} seconds')


Accuracy on the test set: 72.33%
Size of the model: 520.83 KB
CPU usage during inference: 42.65%
Inference time: 5.0864 seconds
